In [ ]:
'''
A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [2]:
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [6]:
help(tf.placeholder)

Help on function placeholder in module tensorflow.python.ops.array_ops:

placeholder(dtype, shape=None, name=None)
    Inserts a placeholder for a tensor that will be always fed.
    
    **Important**: This tensor will produce an error if evaluated. Its value must
    be fed using the `feed_dict` optional argument to `Session.run()`,
    `Tensor.eval()`, or `Operation.run()`.
    
    For example:
    
    ```python
    x = tf.placeholder(tf.float32, shape=(1024, 1024))
    y = tf.matmul(x, x)
    
    with tf.Session() as sess:
      print(sess.run(y))  # ERROR: will fail because x was not fed.
    
      rand_array = np.random.rand(1024, 1024)
      print(sess.run(y, feed_dict={x: rand_array}))  # Will succeed.
    ```
    
    Args:
      dtype: The type of elements in the tensor to be fed.
      shape: The shape of the tensor to be fed (optional). If the shape is not
        specified, you can feed a tensor of any shape.
      name: A name for the operation (optional).
    
    Re

In [9]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out
help(tf.nn.conv2d)

Help on function conv2d in module tensorflow.python.ops.gen_nn_ops:

conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)
    Computes a 2-D convolution given 4-D `input` and `filter` tensors.
    
    Given an input tensor of shape `[batch, in_height, in_width, in_channels]`
    and a filter / kernel tensor of shape
    `[filter_height, filter_width, in_channels, out_channels]`, this op
    performs the following:
    
    1. Flattens the filter to a 2-D matrix with shape
       `[filter_height * filter_width * in_channels, output_channels]`.
    2. Extracts image patches from the input tensor to form a *virtual*
       tensor of shape `[batch, out_height, out_width,
       filter_height * filter_width * in_channels]`.
    3. For each patch, right-multiplies the filter matrix and the image patch
       vector.
    
    In detail, with the default NHWC format,
    
        output[b, i, j, k] =
            sum_{di, dj, q} input[b, strides[1] * i + 

In [8]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))###############????
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
help(tf.reduce_mean)

Help on function reduce_mean in module tensorflow.python.ops.math_ops:

reduce_mean(input_tensor, axis=None, keep_dims=False, name=None, reduction_indices=None)
    Computes the mean of elements across dimensions of a tensor.
    
    Reduces `input_tensor` along the dimensions given in `axis`.
    Unless `keep_dims` is true, the rank of the tensor is reduced by 1 for each
    entry in `axis`. If `keep_dims` is true, the reduced dimensions
    are retained with length 1.
    
    If `axis` has no entries, all dimensions are reduced, and a
    tensor with a single element is returned.
    
    For example:
    
    ```python
    # 'x' is [[1., 1.]
    #         [2., 2.]]
    tf.reduce_mean(x) ==> 1.5
    tf.reduce_mean(x, 0) ==> [1.5, 1.5]
    tf.reduce_mean(x, 1) ==> [1.,  2.]
    ```
    
    Args:
      input_tensor: The tensor to reduce. Should have numeric type.
      axis: The dimensions to reduce. If `None` (the default),
        reduces all dimensions.
      keep_dims: If true, 

In [5]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 1280, Minibatch Loss= 23236.402344, Training Accuracy= 0.34375
Iter 2560, Minibatch Loss= 9403.296875, Training Accuracy= 0.60938
Iter 3840, Minibatch Loss= 5457.389648, Training Accuracy= 0.72656
Iter 5120, Minibatch Loss= 5150.375977, Training Accuracy= 0.78906
Iter 6400, Minibatch Loss= 4373.975098, Training Accuracy= 0.80469
Iter 7680, Minibatch Loss= 4502.446289, Training Accuracy= 0.84375
Iter 8960, Minibatch Loss= 2458.708496, Training Accuracy= 0.89062
Iter 10240, Minibatch Loss= 4538.303711, Training Accuracy= 0.85938
Iter 11520, Minibatch Loss= 1793.116211, Training Accuracy= 0.90625
Iter 12800, Minibatch Loss= 3992.675537, Training Accuracy= 0.87500
Iter 14080, Minibatch Loss= 1738.894287, Training Accuracy= 0.90625
Iter 15360, Minibatch Loss= 1905.705200, Training Accuracy= 0.92188
Iter 16640, Minibatch Loss= 1493.844238, Training Accuracy= 0.90625
Iter 17920, Minibatch Loss= 1173.640503, Training Accuracy= 0.89062
Iter 19200, Minibatch Loss= 1431.767090, Training Accu

Iter 157440, Minibatch Loss= 7.528458, Training Accuracy= 0.99219
Iter 158720, Minibatch Loss= 552.193848, Training Accuracy= 0.95312
Iter 160000, Minibatch Loss= 188.803192, Training Accuracy= 0.96094
Iter 161280, Minibatch Loss= 139.948212, Training Accuracy= 0.97656
Iter 162560, Minibatch Loss= 72.842072, Training Accuracy= 0.97656
Iter 163840, Minibatch Loss= 182.867889, Training Accuracy= 0.97656
Iter 165120, Minibatch Loss= 374.781708, Training Accuracy= 0.95312
Iter 166400, Minibatch Loss= 27.201904, Training Accuracy= 0.99219
Iter 167680, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 168960, Minibatch Loss= 0.000000, Training Accuracy= 1.00000
Iter 170240, Minibatch Loss= 63.592255, Training Accuracy= 0.99219
Iter 171520, Minibatch Loss= 287.482666, Training Accuracy= 0.96094
Iter 172800, Minibatch Loss= 311.781525, Training Accuracy= 0.96875
Iter 174080, Minibatch Loss= 409.390137, Training Accuracy= 0.96875
Iter 175360, Minibatch Loss= 399.688293, Training Accurac